In [1]:
import io

import netaddr


import aiohttp

from rpki_analysis.delegated_stats import read_delegated_extended_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df_extended = read_delegated_extended_stats(io.StringIO(await resp.text()))

In [3]:
def to_cidrs(res: netaddr.IPRange | netaddr.IPNetwork):
    if isinstance(res, netaddr.IPRange):
        return list([str(x) for x in res.cidrs()])
    elif isinstance(res, netaddr.IPNetwork):
            return [str(res)]
    else:
        return [res]

In [4]:
df_extended['resource'] = df_extended.resource.map(to_cidrs)
df_extended = df_extended.explode("resource")

,rir,country,afi,raw_resource,length,date,status,opaque_id,category,resource
7,ripencc,EU,asn,7,1,1993-09-01 00:00:00+00:00,assigned,12cf5dd8-0c74-4405-bcf5-0df45f7dde12,e-stats,7
28,ripencc,DE,asn,28,1,1993-09-01 00:00:00+00:00,assigned,822283a9-e9f2-4155-9408-2b30b469547b,e-stats,28
137,ripencc,IT,asn,137,1,1993-09-01 00:00:00+00:00,assigned,79a53d4d-4e5c-4585-b952-2a936cce5e0b,e-stats,137
224,ripencc,NO,asn,224,1,1993-09-01 00:00:00+00:00,assigned,f5d1ffe0-7d97-4d48-8ebf-87de56e72b1c,e-stats,224
248,ripencc,EU,asn,248,1,1993-09-01 00:00:00+00:00,assigned,3903dde7-01a4-47b2-9c35-57c1e9de0c9c,e-stats,248
...,...,...,...,...,...,...,...,...,...,...
719037,ripencc,ZZ,ipv6,2a14:fd00::,24,2024-12-22 00:00:00+00:00,available,ripencc,e-stats,2a14:fd00::/24
719038,ripencc,ZZ,ipv6,2a14:fe00::,23,2024-12-22 00:00:00+00:00,available,ripencc,e-stats,2a14:fe00::/23
719039,ripencc,ZZ,ipv6,2a15::,16,2024-12-22 00:00:00+00:00,available,ripencc,e-stats,2a15::/16
719040,ripencc,ZZ,ipv6,2a16::,15,2024-12-22 00:00:00+00:00,available,ripencc,e-stats,2a16::/15


In [5]:
df_extended.to_parquet('nro-delegated-stats-20241105.parquet', index=False)